In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import ast
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import numpy as np
import xarray as xr
from datetime import datetime

# tk libs
from lib.objs.tkpaths import Site
from lib.objs.predictor import Predictor
from lib.io.matlab import ReadGowMat
from lib.waves import Calculate_TWL



# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_est_pred = ST.ESTELA.pred_slp        # estela slp predictor

p_wvs_parts = ST.WAVES.partitions_p1  # original wave partitions (hs, tp)
p_wvs_fams_noTCs = ST.WAVES.families_notcs    # wave families (TCs removed)


# output files
p_WVS_MaxStorm = ST.EXTREMES.waves_maxstorm  # waves families at max storm TWL
p_KMA_MaxStorm = ST.EXTREMES.kma_maxstorm    # KMA at max storm TWL


In [2]:
# --------------------------------------
# Load data

# load ESTELA predictor KMA 
pred = Predictor(p_est_pred)
pred.Load()
xds_KMA = pred.KMA

# load original wave partitions (hourly data)
xds_wvs_pts = ReadGowMat(p_wvs_parts)

# load wave families (sea, swl1, swl2) without TCs
xds_wvs_fams_noTCs = xr.open_dataset(p_wvs_fams_noTCs)



# TODO: QUITAR
# --------------------------------------
# Load test data

# TESTEAR CON DATOS MATLAB
from lib.io.matlab import ReadMatfile
from lib.custom_dateutils import DateConverter_Mat2Py as dmp
from lib.custom_dateutils import datevec2datetime as d2d

# load test KMA (bmus, time, number of centers, cenEOFs)
p_test_mat = '/Users/nico/Projects/TESLA-kit/source/teslakit/data/tests/test_ExtremesGEV/Nico_Montecarlo/'
dmatf = ReadMatfile(op.join(p_test_mat, 'bmus_testearpython.mat'))
xds_KMA = xr.Dataset(
    {
        'bmus':('time', dmatf['KMA']['bmus']),
        'cenEOFs':(('n_clusters','n_features',), dmatf['KMA']['cenEOFs']),
        
    },
    coords = {'time':np.array(d2d(dmatf['KMA']['Dates']))}
)
n_clusters = len(dmatf['KMA']['order'])

# load test wave families (sea, swl1, swl2 (Hs, Tp, Dir), time)
dmatf = ReadMatfile(op.join(p_test_mat, 'KWA_waves_2PART_TCs_nan.mat'))
xds_wvs_fam_noTCs = xr.Dataset(
    {
        'sea_Hs':('time', dmatf['sea']['Hs']),
        'sea_Tp':('time', dmatf['sea']['Tp']),
        'sea_Dir':('time', dmatf['sea']['Dir']),
        'swell_1_Hs':('time', dmatf['swl1']['Hs']),
        'swell_1_Tp':('time', dmatf['swl1']['Tp']),
        'swell_1_Dir':('time', dmatf['swl1']['Dir']),
        'swell_2_Hs':('time', dmatf['swl2']['Hs']),
        'swell_2_Tp':('time', dmatf['swl2']['Tp']),
        'swell_2_Dir':('time', dmatf['swl2']['Dir']),
        
    },
    coords = {'time': dmp(dmatf['time'])},
)


# TODO: aqui perdemos datos TWL maximo (al pasar los datos de 3h a 1d), y estamos con extremos....
# PERO para coordinar con bmus hay que trabajar en diarios...
# Select data at KMA time
xds_wvs_fam_noTCs = xds_wvs_fam_noTCs.sel(time=xds_KMA.time)
xds_wvs_pts = xds_wvs_pts.sel(time=xds_KMA.time)


# TODO: DATOS TEST MATLAB, BUSCAR/ORDENAR LOS REALES
# load Simulation data: dates simulation, bmus simulation, MJO (categ, rmm1, rmm2), AWT (PCs123, bmus)
dmatf = ReadMatfile(op.join(p_test_mat, 'DWT_1000years_mjo_awt_v2.mat'))
xds_DWT_MJO_AWT = xr.Dataset(
    {
        'daily_WT':(('n_sims','time',), dmatf['bmusim']),
        
    },
    coords = {'time': dmp(dmatf['datesim'].astype(float))},
)


In [3]:
# --------------------------------------
# Calculate Max TWL for each storm

# TODO: def function

# calculate total water level from waves data
xda_TWL = Calculate_TWL(xds_wvs_pts.hs, xds_wvs_pts.tp)

# locate dates where KMA WT changes (bmus series)
bmus_diff = np.diff(xds_KMA.bmus.values)
ix_ch = np.where((bmus_diff != 0))[0]+1
ix_ch = np.insert(ix_ch, 0,0)  
ds_ch = xds_KMA.time.values[ix_ch]  # dates where WT changes

# list of tuples with (date start, date end) for each WT window
dates_tup_WT = [(ds_ch[c], ds_ch[c+1]-np.timedelta64(1,'D')) for c in range(len(ds_ch)-1)]
dates_tup_WT.append((dates_tup_WT[-1][1]+np.timedelta64(1,'D'), xds_KMA.time.values[-1]))


# find max TWL inside each WT window and store wave families data
TWL_WT_max = []
times_WT_max = []
for d1, d2 in dates_tup_WT:
    
    # get TWL inside WT window
    wt_TWL = xda_TWL.sel(time=slice(d1,d2))[:] 
    
    # get window maximum TWL date
    wt_max_TWL = wt_TWL.where(wt_TWL==wt_TWL.max(), drop=True).squeeze()
    max_TWL = wt_max_TWL.values
    max_date = wt_max_TWL.time.values
        
    # append data
    TWL_WT_max.append(max_TWL)
    times_WT_max.append(max_date)
    
# select waves families data at WT max TWL instants
xds_WVS_MaxStorm = xds_wvs_fam_noTCs.sel(time=times_WT_max)
xds_WVS_MaxStorm['max_TWL'] = ('time', TWL_WT_max)
print(xds_WVS_MaxStorm)

# select kma at WT max TWL instants
xds_KMA_MaxStorm = xds_KMA.sel(time=times_WT_max)
print(xds_KMA_MaxStorm)

# store data for extremes Montecarlo simulation
xds_WVS_MaxStorm.to_netcdf(p_WVS_MaxStorm)
xds_KMA_MaxStorm.to_netcdf(p_KMA_MaxStorm)


<xarray.Dataset>
Dimensions:      (time: 6527)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-22 1979-01-23 ... 2016-12-27
Data variables:
    sea_Hs       (time) float64 0.07675 0.2218 nan nan ... nan 0.4055 0.0714
    sea_Tp       (time) float64 3.593 5.279 nan nan ... 3.554 nan 8.607 3.947
    sea_Dir      (time) float64 154.1 152.3 nan nan ... 132.7 nan 130.9 155.6
    swell_1_Hs   (time) float64 0.7545 1.045 0.7774 ... 0.5823 0.9433 0.788
    swell_1_Tp   (time) float64 12.39 12.8 13.57 12.36 ... 12.79 11.48 11.53
    swell_1_Dir  (time) float64 332.5 332.4 349.8 348.5 ... 2.536 3.501 358.5
    swell_2_Hs   (time) float64 2.825 3.079 2.524 1.812 ... 3.336 2.696 2.331
    swell_2_Tp   (time) float64 9.993 9.377 9.595 9.11 ... 8.238 8.36 9.482 9.51
    swell_2_Dir  (time) float64 65.3 68.52 67.25 68.16 ... 60.52 70.03 64.44
    max_TWL      (time) float64 0.7318 0.8098 0.7329 ... 0.7256 0.7005 0.6512
<xarray.Dataset>
Dimensions:  (n_clusters: 36, n_features: 32, time: 6